# Generate summary from articles
This notebook demonstrates how easy it is to leverage pre-trained models from the Hugging Face Transformers library for tasks like summarization, without having to train the model from scratch.

We use the !pip install transformers command to install the Transformers library, which provides access to pre-trained models and utilities for various NLP tasks.

In [2]:
!pip install transformers huggingface_hub --quiet

This step is necessary to access the pre-trained models hosted on the Hugging Face Hub. The notebook_login() function will prompt you to enter your Hugging Face token or create a new account.

In [3]:
# Authenticate with Hugging Face Hub
from huggingface_hub import notebook_login

notebook_login()


We import the necessary classes from the Transformers library, specifically BartForConditionalGeneration (the pre-trained summarization model) and BartTokenizer (for tokenizing the input text).

In [4]:
# Import required libraries
from transformers import BartForConditionalGeneration, BartTokenizer

We load the pre-trained bart-large-cnn model and tokenizer using the from_pretrained method, specifying the model name

In [5]:
# Load the pre-trained model and tokenizer
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

We provide an example article. You may use other articles to test it out.

In [6]:
# Example article to summarize
article = """
The Astonishing Hypothesis is a 1994 book by scientist Francis Crick about consciousness and neuroscience. In it, Crick promotes the idea that consciousness is produced by physical and chemical processes in the brain, and that neuroscience will eventually have a theory which can explain consciousness. The book surveys the history of research into consciousness and outlines several hypotheses about the neural correlates of various components and properties of consciousness.
"""


We use the tokenizer to tokenize the article text, passing the return_tensors="pt" argument to get the tokenized output as a PyTorch tensor. 

We also set max_length=1024 to truncate the input if it exceeds that length, and truncation=True to allow truncation.

In [7]:
# Tokenize the article
inputs = tokenizer(article, return_tensors="pt", max_length=1024, truncation=True)



When we call the generate method of the pre-trained summarization model, we're asking the model to take the tokenized input (the article text) and generate a summarized output sequence (the summary text).
We pass several arguments to the generate method to control the behavior of the text generation process:

- num_beams=4:

This argument specifies that we want to use beam search for generating the output sequence.
Beam search is a technique used in sequence generation tasks, where the model keeps track of the k most probable sequences at each step, instead of just the single most probable sequence.
In this case, we set num_beams=4, which means the model will consider the 4 most probable sequences at each step when generating the summary.
This helps the model explore more diverse and potentially better summaries, rather than just sticking to the single most likely path.


- max_length=100:

This argument sets the maximum length of the generated summary to 100 tokens.
The model will continue generating tokens (words) for the summary until it either reaches the specified max_length or generates the end-of-sequence token, whichever comes first.
Setting a maximum length helps prevent the model from generating unreasonably long or never-ending summaries.


- early_stopping=True:

This argument tells the model to stop generating the summary as soon as it produces the end-of-sequence token, even if the generated sequence is shorter than the max_length.
End-of-sequence tokens are special tokens that indicate the end of a sequence, similar to a period at the end of a sentence.

By setting early_stopping=True, we allow the model to stop generating the summary once it has produced a complete and coherent sequence, without having to generate up to the max_length if it's not necessary.

In [8]:
# Generate the summary
summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=100, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)


Finally, we print the generated summary

In [9]:
print(f"Summary: {summary}")

Summary: The Astonishing Hypothesis is a 1994 book by scientist Francis Crick about consciousness and neuroscience. In it, Crick promotes the idea that consciousness is produced by physical and chemical processes in the brain, and that neuroscience will eventually have a theory which can explain consciousness.
